In [8]:
import RAG_utils
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
! ollama pull phi3

pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠦ pulling manifest 
pulling 4fed7364ee3e...   0% ▕                ▏    0 B/2.3 GB                  pulling manifest 
pulling 4fed7364ee3e...   0% ▕                ▏    0 B/2.3 GB                  pulling manifest 
pulling 4fed7364ee3e...   0% ▕                ▏    0 B/2.3 GB                  pulling manifest 
pulling 4fed7364ee3e...   0% ▕                ▏    0 B/2.3 GB                  pulling manifest 
pulling 4fed7364ee3e...   0% ▕                ▏    0 B/2.3 GB                  pulling manifest 
pulling 4fed7364ee3e...   0% ▕                ▏    0 B/2.3 GB                  pulling manifest 
pulling 4fed7364ee3e...   0% ▕                ▏    0 B/2.3 GB                  pulling manifest 

In [6]:
RAG_utils.list_chroma_collections()

[Collection(name=train_1000_200_all-MiniLM-L6-v2),
 Collection(name=train_full),
 Collection(name=train_1000_150_all-MiniLM-L6-v2),
 Collection(name=test_1000_175_all-MiniLM-L6-v2),
 Collection(name=test),
 Collection(name=rag_demo_collection)]

In [7]:
db = RAG_utils.access_chroma_db(collection_name='train_1000_200_all-MiniLM-L6-v2')

In [9]:
question = 'When was the house at 3524 Redwing Ct, Naperville, IL 60564 last sold and for what price?'

system_message = """You are a helpful assistant. Answer the user's question in one sentence based on the provided context. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. Do NOT start your response with "According to the provided context." """
user_message_template = """Context: {context} Question: {question}"""

documents = db.similarity_search_with_relevance_scores(question, k=5)
context = RAG_utils.format_docs([doc[0] for doc in documents])
user_message = user_message_template.format(context=context, question=question)

answer = RAG_utils.gen_text_ollama(sys_msg=system_message, user_msg=user_message, model='phi3', options={'seed':0, 'temperature':0.01})
print(question)
print(answer)

When was the house at 3524 Redwing Ct, Naperville, IL 60564 last sold and for what price?
I'm unable to provide real estate transaction data. However, you can typically find this information through public records or by contacting a local real estate agent who may have access to such databases.

For Sale
- 3524 Redwing Ct3 beds, 3 baths
1,980 sqft, 6,750 sqft lot, built in 1999

Learn more about forecast calculations or home value predictions.… More Less
For Sale
- 2401 Redwing Ct3 beds, 3 baths
2,800 sqft, 6,750 sqft lot, built in 1999
- 2401 Redwing Ct4 beds, 4 baths
3,200 sqft, 6,750 sqft lot, built in 1999
- 2401 Redwing Ct5 beds, 5 baths
3,800 sqft, 6,750 sqft lot, built in 1999
- 2401 Redwing Ct6 beds, 6 baths
4,400 sqft, 6,750 sqft lot, built in 1999
- 2401 Redwing Ct7 beds, 7 baths
5,000 sqft, 6,750 sqft lot, built in 1999

Learn more about forecast calculations or home value predictions.… More Less
For Sale
- 2401 Redwing Ct3 beds, 3 baths
2,800 sqft, 6,750 sqft lot, built in 

In [11]:
# rerank documents
question = 'When was the house at 3524 Redwing Ct, Naperville, IL 60564 last sold and for what price?'

system_message = """You are a helpful assistant. Answer the user's question in one sentence based on the provided context. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. Do NOT start your response with "According to the provided context." """
user_message_template = """Context: {context} Question: {question}"""

documents = db.similarity_search_with_relevance_scores(question, k=5)
documents[0], documents[1] = documents[1], documents[0]

print(documents[0][0])
context = RAG_utils.format_docs([doc[0] for doc in documents])
user_message = user_message_template.format(context=context, question=question)

answer = RAG_utils.gen_text_ollama(sys_msg=system_message, user_msg=user_message, model='phi3', options={'seed':0, 'temperature':0.01})
print(question)
print(answer)

page_content="3524 Redwing Ct, Naperville, IL 60564\n4 beds5 baths3,591 sqft Edit\nA Zestimate® home valuation is Zillow's estimated market value. It is not an appraisal. Use it as a starting point to determine a home's value. Learn more\nFacts\n- Single Family\n- Built in 2000\n- Views: 773 all time views\n- Cooling: Central, Other\n- Heating: Forced air, Other\n- Last sold: Oct 2013 for $595,000\n- Last sale price/sqft: $166\nFeatures\n- Ceiling Fan\n- Deck\n- Fireplace\n- Flooring: Carpet, Hardwood\n- Mother-in-Law\n- Parking: Garage - Attached, 3 spaces, 704 sqft\n- Security System\n- Vaulted Ceiling\nAppliances Included\n- Dishwasher\n- Dryer\n- Garbage disposal\n- Microwave\n- Range / Oven\n- Refrigerator\n- Washer\nRoom Types\n- Dining room\n- Family room\n- Office\n- Recreation room\nConstruction\n- Exterior material: Brick\n- Roof type: Asphalt\n- Room count: 11\n- Stories: 2\n- Structure type: Other\n- Unit count: 1\nOther\n- Floor size: 3,591 sqft\n- Heating: Gas\n- Laundry: